# Data prepare

In [1]:
import numpy as np
import pandas as pd
from scipy.spatial.distance import cdist
from scipy.linalg import solve
from math import radians, cos, sin, sqrt, atan2

In [2]:
# Load data
air_data = pd.read_excel('test_air_sensor.xlsx')
wind_data = pd.read_csv('wind.csv')


# Split the "Latitude & Longitude" column into separate columns
air_data[['Latitude', 'Longitude']] = air_data['Latitude & Longitude'].str.split(', ', expand=True)
air_data['Latitude'] = air_data['Latitude'].astype(float)
air_data['Longitude'] = air_data['Longitude'].astype(float)

# definition

In [3]:
# Haversine formula to calculate distance between two points in meters
def haversine(lat1, lon1, lat2, lon2):
    R = 6371000  # Radius of the Earth in meters
    phi1 = radians(lat1)
    phi2 = radians(lat2)
    delta_phi = radians(lat2 - lat1)
    delta_lambda = radians(lon2 - lon1)
    a = sin(delta_phi / 2.0) ** 2 + cos(phi1) * cos(phi2) * sin(delta_lambda / 2.0) ** 2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    meters = R * c  # Output distance in meters
    return meters

def calculate_semivariogram(data):
    num_points = len(data)  # Number of data points
    semivariances = []  # List to store semivariance values

    # Nested loop to compare each pair of points
    for i in range(num_points):
        for j in range(i + 1, num_points):
            # Calculate the distance between point i and point j
            dist = haversine(data['Latitude'].iloc[i], data['Longitude'].iloc[i],
                             data['Latitude'].iloc[j], data['Longitude'].iloc[j])
            # Calculate the squared difference of the NO2 values
            squared_diff = (data['NO2(ug m-3)'].iloc[i] - data['NO2(ug m-3)'].iloc[j]) ** 2
            # Append the distance and squared difference as a tuple to semivariances
            semivariances.append((dist, squared_diff))

    # Calculate the average semivariance for each unique distance
    unique_distances = sorted(set([item[0] for item in semivariances]))
    avg_semivariances = []
    for dist in unique_distances:
        squared_diffs = [item[1] for item in semivariances if item[0] == dist]
        avg_semivariances.append((dist, np.mean(squared_diffs) / 2.0))

    # Return the list of (distance, semivariance) tuples as a numpy array
    return np.array(avg_semivariances)

# # Calculate Kriging weights
# def calculate_kriging_weights(semivariogram, distances, n):
#     A = np.zeros((n + 1, n + 1))
#     A[:n, :n] = semivariogram[distances.astype(int)]
#     A[-1, :-1] = 1
#     A[:-1, -1] = 1

#     b = np.zeros(n + 1)
#     b[:-1] = semivariogram[distances.astype(int)]

#     weights = solve(A, b)
#     return weights[:-1]


# def calculate_kriging_weights(semivariogram, distances, n):
#     A = np.zeros((n + 1, n + 1))
#     for i in range(n):
#         for j in range(n):
#             A[i, j] = semivariogram[int(distances[i, j])]
#     A[-1, :-1] = 1
#     A[:-1, -1] = 1

#     b = np.zeros(n + 1)
#     for i in range(n):
#         b[i] = semivariogram[int(distances[i, -1])]

#     weights = solve(A, b)
#     return weights[:-1]

def calculate_kriging_weights(semivariogram, distances, n, nugget=1e-10):
    A = np.zeros((n + 1, n + 1))
    
    for i in range(n):
        for j in range(n):
            if i == j:
                A[i, j] = semivariogram[0][1] + nugget  # Semivariance at distance 0 with nugget effect
            else:
                dist = int(distances[0, j])
                A[i, j] = semivariogram[dist][1] if dist < len(semivariogram) else semivariogram[-1][1]  # Last semivariogram value for large distances

    A[-1, :-1] = 1
    A[:-1, -1] = 1

    b = np.zeros(n + 1)
    for i in range(n):
        dist = int(distances[0, i])
        b[i] = semivariogram[dist][1] if dist < len(semivariogram) else semivariogram[-1][1]

    weights = solve(A, b)
    return weights[:-1]

# Adjust weights for wind effects
def adjust_weights(weights, wind_speed, wind_dir, sensor_directions, avg_wind_speed):
    adjustments = 1 + (wind_speed * np.cos(np.radians(wind_dir - sensor_directions))) / avg_wind_speed
    adjusted_weights = weights * adjustments
    return adjusted_weights

# Normalize adjusted weights
def normalize_weights(weights):
    return weights / np.sum(weights)

# Perform interpolation
def interpolate(data, weights):
    return np.sum(weights * data['NO2(ug m-3)'].values)

# Example

In [4]:
# Filter data for the specific date
date_filter = '2019-04-01'
filtered_air_data = air_data[air_data['Monitor Date'] == date_filter]
filtered_wind_data = wind_data[wind_data['date'] == date_filter]

# 移除'NO2(ug m-3)'列中的NaN值
filtered_air_data = filtered_air_data.dropna(subset=['NO2(ug m-3)'])
# 排除'NO2(ug m-3)'列中等于0的行
filtered_air_data = filtered_air_data[filtered_air_data['NO2(ug m-3)'] != 0]
# 如果'NO2(ug m-3)'列包含字符串类型的空值，也排除这些行（可选）
# filtered_air_data = filtered_air_data[filtered_air_data['NO2(ug m-3)'] != '']

# Extract wind direction and speed for the specific date
wind_speed = filtered_wind_data['wspd'].values[0]
wind_dir = filtered_wind_data['wdir'].values[0]

# Assume max_distance is 10 (you can change as needed)

semivariogram = calculate_semivariogram(filtered_air_data)

# Calculate the center point of all sensor locations
interpolation_point = np.array([[filtered_air_data['Latitude'].mean(), filtered_air_data['Longitude'].mean()]])
# Distances from interpolation point to sensors
# distances = cdist(interpolation_point, filtered_air_data[['Latitude', 'Longitude']], metric='euclidean')
# Calculate distances from interpolation point to sensors using Haversine formula
sensor_locations = filtered_air_data[['Latitude', 'Longitude']].values
distances = np.array([[haversine(interpolation_point[0, 0], interpolation_point[0, 1], lat, lon) for lat, lon in sensor_locations]])

# Calculate Kriging weights
kriging_weights = calculate_kriging_weights(semivariogram, distances, len(filtered_air_data))

# Adjust weights for wind effects
sensor_directions = np.arctan2(filtered_air_data['Longitude'] - interpolation_point[0, 1], filtered_air_data['Latitude'] - interpolation_point[0, 0]) * 180 / np.pi

mean_wind_speed=wind_data['wspd'].mean()
adjusted_weights = adjust_weights(kriging_weights, wind_speed, wind_dir, sensor_directions, mean_wind_speed)

# Normalize the adjusted weights
normalized_weights = normalize_weights(adjusted_weights)

# Interpolate to get the estimated air quality
estimated_value = interpolate(filtered_air_data, normalized_weights)
print(f'Estimated NO2 at the interpolation point: {estimated_value:.2f} ug/m3')


Estimated NO2 at the interpolation point: 34.39 ug/m3


# debug